In [1]:
import numpy as np
import xarray as xr
import xgcm

In [2]:
### Load dataset
ds = xr.open_zarr('/rigel/ocp/users/jt2796/eccov4r3_output/')

In [3]:
### Creating the grid object
face_connections = {'face':
                    {0: {'X':  ((12, 'Y', False), (3, 'X', False)),
                         'Y':  (None,             (1, 'Y', False))},
                     1: {'X':  ((11, 'Y', False), (4, 'X', False)),
                         'Y':  ((0, 'Y', False),  (2, 'Y', False))},
                     2: {'X':  ((10, 'Y', False), (5, 'X', False)),
                         'Y':  ((1, 'Y', False),  (6, 'X', False))},
                     3: {'X':  ((0, 'X', False),  (9, 'Y', False)),
                         'Y':  (None,             (4, 'Y', False))},
                     4: {'X':  ((1, 'X', False),  (8, 'Y', False)),
                         'Y':  ((3, 'Y', False),  (5, 'Y', False))},
                     5: {'X':  ((2, 'X', False),  (7, 'Y', False)),
                         'Y':  ((4, 'Y', False),  (6, 'Y', False))},
                     6: {'X':  ((2, 'Y', False),  (7, 'X', False)),
                         'Y':  ((5, 'Y', False),  (10, 'X', False))},
                     7: {'X':  ((6, 'X', False),  (8, 'X', False)),
                         'Y':  ((5, 'X', False),  (10, 'Y', False))},
                     8: {'X':  ((7, 'X', False),  (9, 'X', False)),
                         'Y':  ((4, 'X', False),  (11, 'Y', False))},
                     9: {'X':  ((8, 'X', False),  None),
                         'Y':  ((3, 'X', False),  (12, 'Y', False))},
                     10: {'X': ((6, 'Y', False),  (11, 'X', False)),
                          'Y': ((7, 'Y', False),  (2, 'X', False))},
                     11: {'X': ((10, 'X', False), (12, 'X', False)),
                          'Y': ((8, 'Y', False),  (1, 'X', False))},
                     12: {'X': ((11, 'X', False), None),
                          'Y': ((9, 'Y', False),  (0, 'X', False))}}}

grid = xgcm.Grid(ds, face_connections=face_connections)

In [4]:
### Define terms
# Before doing the budget calculations we need to define some terms that will be used in the budget calculations.

### Number of seconds between each snapshot
# There are no snapshots for the first and last time point. Thus, we are skipping budget calculations for first and last month of the given time period.
dt = ds.time_snp[1:].load()

# delta t in seconds. Note: devide by 10**9 to convert nanoseconds to seconds
dt.values = [float(t)/10**9 for t in np.diff(ds.time_snp)]

# Rename time (and iter) axis
dt = dt.rename({'time_snp':'time','iter_snp':'iter'})

# time axis of dt should be the same as of the monthly averages
dt.time.values = ds.time[1:-1].values

In [5]:
### Relevant constants
# Density kg/m^3
rhoconst = 1029

# Heat capacity (J/kg/K)
c_p = 3994

# Constants for surface heat penetration (from Table 2 of Paulson and Simpson, 1977)
R = 0.62
zeta1 = 0.6
zeta2 = 20.0

In [6]:
### Ocean depth
Depth = ds.Depth

### Grid dimensions
dxG = ds.dxG
dyG = ds.dyG
drF = ds.drF
rA = ds.rA
hFacC = ds.hFacC.load()

# Volume (m^3)
vol = (rA*drF*hFacC).transpose('face','k','j','i')

In [7]:
### Land masks

# Make copy of hFacC
mskC = hFacC.copy(deep=True).load()

# Change all fractions (ocean) to 1. land = 0
mskC.values[mskC.values>0] = 1

# Make 2D land mask for surface (This is just for plotting/mapping purposes)
land_mask = mskC[0]
land_mask.values[land_mask.values==0] = np.nan

# Make 3D land mask
land_mask_3d = mskC.copy()
land_mask_3d.values[land_mask_3d.values==0] = np.nan

In [8]:
### Evaluating the volume budget
### Total tendency
# - ETAN: Surface Height Anomaly (m)

# Load snapshots for surface height anomaly from dataset
ETANsnp = ds.ETAN_snp

# Total tendency (1/s)
tendV = xr.DataArray(50*[1],coords={'k': np.array(range(0,50))},dims=['k'])*grid.diff(ETANsnp, 'T', boundary='fill', 
                                                                                      fill_value=0.0)/Depth/dt
tendV['Z'] = ds.Z
tendV['hFacC'] = ds.hFacC
tendV = xr.concat([np.nan*xr.zeros_like(vol).assign_coords(time=ds.time[0]).expand_dims('time'),tendV,
                   np.nan*xr.zeros_like(vol).assign_coords(time=ds.time[-1]).expand_dims('time')],
                  dim='time').transpose('time','face','k','j','i')

In [9]:
### Forcing
# - oceFWflx: net surface Fresh-Water flux into the ocean (kg/m^2/s)

# Load monthly averaged freshwater flux
oceFWflx = ds.oceFWflx.assign_coords(k=0).expand_dims('k')

# Sea surface forcing on volume (1/s)
forcV = xr.concat([(oceFWflx/rhoconst)/(hFacC*drF), 
                   xr.zeros_like(((oceFWflx[0]/rhoconst)/(hFacC*drF)).transpose('time','face','k','j','i'))[:,:,1:]], 
                  dim='k').transpose('time','face','k','j','i')*land_mask_3d.transpose('face','k','j','i')

In [10]:
# ### Horizontal convergence
# - UVELMASS: U Mass-Weighted Comp of Velocity (m/s)
# - VVELMASS: V Mass-Weighted Comp of Velocity (m/s)

# Load monthly averaged velocities
UVELMASS = ds.UVELMASS
VVELMASS = ds.VVELMASS

# Horizontal volume transports (m^3/s)
u_transport = UVELMASS * dyG * drF
v_transport = VVELMASS * dxG * drF

uv_diff = grid.diff_2d_vector({'X' : u_transport, 'Y' : v_transport}, boundary = 'fill')

u_diffx = uv_diff['X']
v_diffy = uv_diff['Y']

# Convergence of the horizontal flow (1/s)
hConvV = (-(u_diffx + v_diffy)/vol).transpose('time','face','k','j','i')

In [11]:
# ### Vertical convergence
# - WVELMASS: Vertical Mass-Weighted Comp of Velocity (m/s)

# Load monthly averaged vertical velocity
WVELMASS = ds.WVELMASS.transpose('time','face','k_l','j','i')

# Vertical volume transport (m^3/s)
w_transport = (WVELMASS * rA).transpose('time','face','k_l','j','i')

# Apparently, it is required to add the vertical volume flux at the air-sea interface (`oceFWflx`) to the surface layer to balance the budget.
# Add the vertical volume flux at the air-sea interface
w_transport = xr.concat([(w_transport.sel(k_l=0) + (forcV*vol).sel(k=0)).expand_dims('k_l').drop(['k','Z','PHrefC','drF','hFacC']),
                         w_transport[:,:,1:]], dim='k_l').transpose('time','face','k_l','j','i')

# Convergence of the vertical flow (1/s)
vConvV = grid.diff(w_transport, 'Z', boundary='fill')/vol

In [12]:
### Evaluating the heat budget

# ### Total tendency
# - THETA: Potential Temperature (degC)

# Load snapshots of theta
THETAsnp = ds.THETA_snp

# Calculate the s∗theta term
HCsnp = (THETAsnp*(1+ETANsnp/Depth)).transpose('time_snp','face','k','j','i')

# Total tendency (degC/s)
tendH = grid.diff(HCsnp, 'T', boundary='fill', fill_value=0.0)/dt

# Add coordinates
tendH['Depth'] = ds.Depth
tendH['XC'] = ds.XC
tendH['YC'] = ds.YC
tendH['Z'] = ds.Z
tendH['rA'] = ds.rA
tendH['PHrefC'] = ds.PHrefC
tendH['drF'] = ds.drF
tendH['hFacC'] = ds.hFacC
tendH = xr.concat([np.nan*xr.zeros_like(vol).assign_coords(time=ds.time[0]).expand_dims('time'),tendH,
                   np.nan*xr.zeros_like(vol).assign_coords(time=ds.time[-1]).expand_dims('time')],
                  dim='time').transpose('time','face','k','j','i')

In [13]:
### Forcing
# - TFLUX: total heat flux (match heat-content variations) (W/m^2)
# - oceQsw: net Short-Wave radiation (+=down) (W/m^2)

# Load monthly averages of heat flux and shortwave radiation
TFLUX = ds.TFLUX
oceQsw = ds.oceQsw

#### Defining terms needed for evaluating surface heat forcing
Z = ds.Z.load()
RF = np.concatenate([ds.Zp1.values[:-1],[np.nan]])

# **Note**: `Z` and `Zp1` are used in deriving surface heat penetration. MATLAB code uses `RF` from `mygrid` structure.

q1 = R*np.exp(1.0/zeta1*RF[:-1]) + (1.0-R)*np.exp(1.0/zeta2*RF[:-1])
q2 = R*np.exp(1.0/zeta1*RF[1:]) + (1.0-R)*np.exp(1.0/zeta2*RF[1:])

# Correction for the 200m cutoff
zCut = np.where(Z < -200)[0][0]
q1[zCut:] = 0
q2[zCut-1:] = 0

# Save q1 and q2 as xarray data arrays
q1 = xr.DataArray(q1,coords=[Z.k],dims=['k'])
q2 = xr.DataArray(q2,coords=[Z.k],dims=['k'])


#### Compute vertically penetrating flux

# Surface heat flux (below the surface)
forcH_subsurf = ((q1*(mskC==1)-q2*(mskC.shift(k=-1)==1))*oceQsw).transpose('time','face','k','j','i')

# Surface heat flux (at the sea surface)
forcH_surf = ((TFLUX - (1-(q1[0]-q2[0]))*oceQsw)*mskC[0]).transpose('time','face','j','i').assign_coords(k=0).expand_dims('k')

# Full-depth forcing
forcH = xr.concat([forcH_surf,forcH_subsurf[:,:,1:]], dim='k').transpose('time','face','k','j','i')

# #### Add geothermal heat flux
# Geothermal flux needs to be a three dimensional field since the sources are distributed along the ocean floor at various depths. This requires a three dimensional mask.

# Create 3d bathymetry mask
mskC_shifted = mskC.shift(k=-1)

mskC_shifted.values[-1,:,:,:] = 0
mskb = mskC - mskC_shifted

# Create 3d field of geothermal heat flux
geoflx3d = ds.GEOFLX * mskb.transpose('face','k','j','i')
GEOFLX = geoflx3d.transpose('face','k','j','i')

# Add geothermal heat flux to forcing field and convert from W/m^2 to degC/s
forcH = ((forcH + GEOFLX)/(rhoconst*c_p))/(hFacC*drF)

In [17]:
# ### Advection
# #### Horizontal convergence
# - ADVx_TH: U Comp. Advective Flux of Pot.Temperature (degC m^3/s)
# - ADVy_TH: V Comp. Advective Flux of Pot.Temperature (degC m^3/s)

ADVx_TH = ds.ADVx_TH
ADVy_TH = ds.ADVy_TH

ADVxy_diff = grid.diff_2d_vector({'X' : ADVx_TH, 'Y' : ADVy_TH}, boundary = 'fill')

ADVx_diffx = ADVxy_diff['X']
ADVy_diffy = ADVxy_diff['Y']

# Convergence of horizontal advection (degC/s)
adv_hConvH = (-(ADVx_diffx + ADVy_diffy)/vol).transpose('time','face','k','j','i')

In [18]:
# #### Vertical convergence
# - ADVr_TH: Vertical Advective Flux of Pot.Temperature (degC m^3/s)

# Load monthly averages of vertical advective flux
ADVr_TH = ds.ADVr_TH.transpose('time','face','k_l','j','i')

# For `ADVr_TH`, `DFrE_TH` and `DFrI_TH`, we need to make sure that sequence of dimensions are consistent. 
# When loading the fields use `.transpose('time','face','k_l','j','i')`.
#Otherwise, the divergences will be not correct (at least for face = 12).

# Convergence of vertical advection (degC/s)
adv_vConvH = (grid.diff(ADVr_TH, 'Z', boundary='fill')/vol).transpose('time','face','k','j','i')

In [19]:
# ### Diffusion
# #### Horizontal convergence
# - DFxE_TH: U Comp. Diffusive Flux of Pot.Temperature (degC m^3/s)
# - DFyE_TH: V Comp. Diffusive Flux of Pot.Temperature (degC m^3/s)

# Load monthly averages of diffusive fluxes
DFxE_TH = ds.DFxE_TH
DFyE_TH = ds.DFyE_TH

DFxyE_diff = grid.diff_2d_vector({'X' : DFxE_TH, 'Y' : DFyE_TH}, boundary = 'fill')

DFxE_diffx = DFxyE_diff['X']
DFyE_diffy = DFxyE_diff['Y']

# Convergence of horizontal diffusion (degC/s)
dif_hConvH = (-(DFxE_diffx + DFyE_diffy)/vol).transpose('time','face','k','j','i')

In [20]:
# #### Vertical convergence
# - DFrE_TH: Vertical Diffusive Flux of Pot.Temperature (Explicit part) (degC m^3/s)
# - DFrI_TH: Vertical Diffusive Flux of Pot.Temperature (Implicit part) (degC m^3/s)

# Load monthly averages of vertical diffusive fluxes
DFrE_TH = ds.DFrE_TH.transpose('time','face','k_l','j','i')
DFrI_TH = ds.DFrI_TH.transpose('time','face','k_l','j','i')

# Convergence of vertical diffusion (degC/s)
dif_vConvH = ((grid.diff(DFrE_TH, 'Z', boundary='fill') + grid.diff(DFrI_TH, 'Z', boundary='fill'))/vol).transpose('time','face','k','j','i')

In [21]:
# ## Evaluating the salt budget

# ### Total tendency
# - SALT: Salinity (psu)

# Load salinity snapshot
SALTsnp = ds.SALT_snp

# Calculate s*S term
sSALT = (SALTsnp*(1+ETANsnp/Depth)).transpose('time_snp','face','k','j','i')

# Total tendency (psu/s)
tendS = grid.diff(sSALT, 'T', boundary='fill', fill_value=0.0)/dt

# Add coordinates
tendS['Depth'] = ds.Depth
tendS['XC'] = ds.XC
tendS['YC'] = ds.YC
tendS['Z'] = ds.Z
tendS['rA'] = ds.rA
tendS['PHrefC'] = ds.PHrefC
tendS['drF'] = ds.drF
tendS['hFacC'] = ds.hFacC
tendS = xr.concat([np.nan*xr.zeros_like(vol).assign_coords(time=ds.time[0]).expand_dims('time'),tendS,
                   np.nan*xr.zeros_like(vol).assign_coords(time=ds.time[-1]).expand_dims('time')],
                  dim='time').transpose('time','face','k','j','i')

In [22]:
# ### Forcing
# - SFLUX: total salt flux (match salt-content variations) (g/m^2/s)
# - oceSPtnd: salt tendency due to salt plume flux (g/m^2/s)

# Load monthly averaged SFLUX and oceSPtnd
SFLUX = ds.SFLUX.assign_coords(k=0).expand_dims('k')
oceSPtnd = ds.oceSPtnd.transpose('time','face','k','j','i')

# `SFLUX` and `oceSPtnd` is given in g/m^2/s.
# Dividing by density and corresponding vertical length scale (`drF`) results in g/kg/s, which is the same as psu/s.

# Surface salt flux (psu/s)
forcS_surf = ((SFLUX/rhoconst)/(hFacC*drF)).transpose('time','face','k','j','i')

# Salt tendency (psu/s)
forcS_tend = ((oceSPtnd/rhoconst)/(hFacC*drF)).transpose('time','face','k','j','i')

forcS = xr.concat([forcS_surf+forcS_tend,forcS_tend[:,:,1:]], dim='k').transpose('time','face','k','j','i')

In [23]:
# ### Advection
# #### Horizontal convergence
# - ADVx_SLT: U Comp. Advective Flux of Salinity (psu m^3/s)
# - ADVy_SLT: V Comp. Advective Flux of Salinity (psu m^3/s)

# Load monthly averaged advective fluxes
ADVx_SLT = ds.ADVx_SLT
ADVy_SLT = ds.ADVy_SLT

ADVxy_SLT_diff = grid.diff_2d_vector({'X' : ADVx_SLT, 'Y' : ADVy_SLT}, boundary = 'fill')

ADVx_SLT_diffx = ADVxy_SLT_diff['X']
ADVy_SLT_diffy = ADVxy_SLT_diff['Y']

# Convergence of horizontal advection (psu/s)
adv_hConvS = (-(ADVx_SLT_diffx + ADVy_SLT_diffy)/vol).transpose('time','face','k','j','i')

In [24]:
# #### Vertical convergence
# - ADVr_SLT: Vertical Advective Flux of Salinity (psu m^3/s)

# Load monthly averaged vertical advective flux
ADVr_SLT = ds.ADVr_SLT.transpose('time','face','k_l','j','i')

# The salt budget only balances when the sea surface forcing is not added to the vertical salt flux 
#(at the air-sea interface). This is different from the volume and salinity budget.

# Convergence of vertical advection (psu/s)
adv_vConvS = (grid.diff(ADVr_SLT, 'Z', boundary='fill')/vol).transpose('time','face','k','j','i')

In [25]:
# ### Diffusion
# #### Horizontal convergence
# - DFxE_SLT: U Comp. Diffusive Flux of Salinity (psu m^3/s)
# - DFyE_SLT: V Comp. Diffusive Flux of Salinity (psu m^3/s)

# Load monthly averaged horizontal diffusive fluxes
DFxE_SLT = ds.DFxE_SLT
DFyE_SLT = ds.DFyE_SLT

DFxyE_SLT_diff = grid.diff_2d_vector({'X' : DFxE_SLT, 'Y' : DFyE_SLT}, boundary = 'fill')

DFxE_SLT_diffx = DFxyE_SLT_diff['X']
DFyE_SLT_diffy = DFxyE_SLT_diff['Y']

# Convergence of horizontal diffusion (psu/s)
dif_hConvS = (-(DFxE_SLT_diffx + DFyE_SLT_diffy)/vol).transpose('time','face','k','j','i')

In [26]:
# #### Vertical convergence
# - DFrE_SLT: Vertical Diffusive Flux of Salinity (Explicit part) (psu m^3/s)
# - DFrI_SLT: Vertical Diffusive Flux of Salinity (Implicit part) (psu m^3/s)

# Load monthly averaged vertical diffusive fluxes
DFrE_SLT = ds.DFrE_SLT.transpose('time','face','k_l','j','i')
DFrI_SLT = ds.DFrI_SLT.transpose('time','face','k_l','j','i')

# Convergence of vertical diffusion (psu/s)
dif_vConvS = ((grid.diff(DFrE_SLT, 'Z', boundary='fill') + grid.diff(DFrI_SLT, 'Z', boundary='fill'))/vol).transpose('time','face','k','j','i')

In [27]:
### Evaluating the salinity budget
#### Scale factor
# - Depth: Ocean_depth (m)
# - ETAN: Surface Height Anomaly (m)

# Load monthly averaged surface height anomaly
ETAN = ds.ETAN

# Scale factor
rstarfac = ((Depth + ETAN)/Depth).transpose('time','face','j','i')

### Total tendency
# Total tendency (psu/s)
tendSln = grid.diff(SALTsnp, 'T', boundary='fill', fill_value=0.0)/dt

# Add coordinates
tendSln['Depth'] = ds.Depth
tendSln['XC'] = ds.XC
tendSln['YC'] = ds.YC
tendSln['Z'] = ds.Z
tendSln['rA'] = ds.rA
tendSln['PHrefC'] = ds.PHrefC
tendSln['drF'] = ds.drF
tendSln['hFacC'] = ds.hFacC

tendSln = xr.concat([np.nan*xr.zeros_like(vol).assign_coords(time=ds.time[0]).expand_dims('time'),tendSln,
                     np.nan*xr.zeros_like(vol).assign_coords(time=ds.time[-1]).expand_dims('time')],
                    dim='time').transpose('time','face','k','j','i')

In [28]:
#### Forcing
# The forcing term is comprised of both salt flux (`forcS`) and volume (i.e., freshwater) flux (`forcV`).
# - SALT: Salinity (psu)
# Load monthly averaged salinity fields
SALT = ds.SALT.transpose('time','face','k','j','i')

### Sea surface forcing for salinity (psu/s)
forcSln = ((-SALT*forcV + forcS)/rstarfac).transpose('time','face','k','j','i')

In [29]:
#### Advection
### Horizontal convergence
adv_hConvSln = ((-SALT*hConvV + adv_hConvS)/rstarfac).transpose('time','face','k','j','i')

### Vertical convergence
adv_vConvSln = ((-SALT*vConvV + adv_vConvS)/rstarfac).transpose('time','face','k','j','i')

#### Diffusion
### Horizontal convergence
dif_hConvSln = dif_hConvS/rstarfac

### Vertical convergence
dif_vConvSln = dif_vConvS/rstarfac

In [30]:
#### Save to dataset

varnames = [
'tendV',
 'forcV',
 'hConvV',
 'vConvV',
 'tendH',
 'forcH',
 'adv_hConvH',
 'adv_vConvH',
 'dif_hConvH',
 'dif_vConvH',
 'tendS',
 'forcS',
 'adv_hConvS',
 'adv_vConvS',
 'dif_hConvS',
 'dif_vConvS',
 'tendSln',
 'forcSln',
 'adv_hConvSln',
 'adv_vConvSln',
 'dif_hConvSln',
 'dif_vConvSln'
]

In [31]:
ds_out = xr.Dataset(data_vars={})
for varname in varnames:
    ds_out[varname] = globals()[varname].chunk(chunks={'time':1,'face':13,'k':50,'j':90,'i':90})

In [32]:
ds_out = ds_out.assign_coords(dt=dt)
ds_out.dt.attrs = {'units': 's','standard_name': 'dt','coordinate': 'time','long_name': 'time span between snapshots'}

In [33]:
ds_out.time.encoding = {}

In [34]:
ds_out

<xarray.Dataset>
Dimensions:       (face: 13, i: 90, j: 90, k: 50, time: 288)
Coordinates:
    Depth         (face, j, i) float32 dask.array<shape=(13, 90, 90), chunksize=(13, 90, 90)>
    XC            (face, j, i) float32 dask.array<shape=(13, 90, 90), chunksize=(13, 90, 90)>
    YC            (face, j, i) float32 dask.array<shape=(13, 90, 90), chunksize=(13, 90, 90)>
  * face          (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i             (i) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...
  * j             (j) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...
    rA            (face, j, i) float32 dask.array<shape=(13, 90, 90), chunksize=(13, 90, 90)>
    PHrefC        (k) float32 dask.array<shape=(50,), chunksize=(50,)>
    Z             (k) float32 dask.array<shape=(50,), chunksize=(50,)>
    drF           (k) float32 dask.array<shape=(50,), chunksize=(50,)>
  * k             (k) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...
    hFacC         (k, f

### Save to zarr

In [35]:
from dask.diagnostics import ProgressBar

In [36]:
with ProgressBar():
    ds_out.to_zarr('/rigel/ocp/users/jt2796/eccov4r3_datasets/eccov4r3_budgets')

/rigel/home/jt2796/miniconda/envs/default/lib/python3.6/site-packages/xarray/backends/api.py:858: SerializationWarning: saving variable None with floating point data as an integer dtype without any _FillValue to use for NaNs
  dataset.dump_to_store(store, sync=True, encoding=encoding, compute=compute)


[                                        ] | 0% Completed |  1min  8.5s

/rigel/home/jt2796/dask/dask/local.py:255: RuntimeWarning: divide by zero encountered in true_divide
  return func(*args2)
/rigel/home/jt2796/dask/dask/local.py:255: RuntimeWarning: invalid value encountered in true_divide
  return func(*args2)


[                                        ] | 0% Completed |  1min  8.9s

/rigel/home/jt2796/dask/dask/local.py:255: RuntimeWarning: invalid value encountered in multiply
  return func(*args2)


[########################################] | 100% Completed | 17min 49.0s


### Check

In [37]:
ds_budg = xr.open_zarr('/rigel/ocp/users/jt2796/eccov4r3_datasets/eccov4r3_budgets')

In [38]:
ds_budg

<xarray.Dataset>
Dimensions:       (face: 13, i: 90, j: 90, k: 50, time: 288)
Coordinates:
    Depth         (face, j, i) float32 dask.array<shape=(13, 90, 90), chunksize=(13, 90, 90)>
    PHrefC        (k) float32 dask.array<shape=(50,), chunksize=(50,)>
    XC            (face, j, i) float32 dask.array<shape=(13, 90, 90), chunksize=(13, 90, 90)>
    YC            (face, j, i) float32 dask.array<shape=(13, 90, 90), chunksize=(13, 90, 90)>
    Z             (k) float32 dask.array<shape=(50,), chunksize=(50,)>
    drF           (k) float32 dask.array<shape=(50,), chunksize=(50,)>
    dt            (time) int64 dask.array<shape=(288,), chunksize=(287,)>
  * face          (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
    hFacC         (k, face, j, i) float32 dask.array<shape=(50, 13, 90, 90), chunksize=(50, 13, 90, 90)>
  * i             (i) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...
    iter          (time) int64 dask.array<shape=(288,), chunksize=(1,)>
  * j             (j) in